In [4]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import random

In [2]:
# frac: fraction of entities to keep
def convert_matscholar_to_bio(filepath, output_path, frac):
    file = open(filepath, 'r')
    content = file.read()
    file.close()
    
    sections = content.split('\n\n')
    df = pd.DataFrame(columns=['sentence', 'word_labels'])
    removed = 0
    kept = 0
        
    for section in sections: 
        sentence = []
        labels = []
        lines = section.split('\n')
        for line in lines:
            if line.strip() == '' or line.strip() == 'O':
                continue
            if len(labels) == 0:
                last_label = 'O'
            else:
                last_label = labels[-1]
            
            try:
                token, label = line.split()[0], line.split()[1][0]
            except:
                print(line.strip())
                
            
            if label not in ['B', 'I', 'O']:
                tokens, label = line.split()[:-1], line.split()[-1][0]
                print(tokens, label)
                for t in tokens:
                    sentence.append(t)
                    labels.append(label)
                continue
            
            p = random.random()
            if label == 'B' and p > frac:
                label = 'O'
                removed += 1
            elif label == 'B' and p < frac:
                kept += 1
            elif label == 'I' and last_label == 'O':
                label = 'O'
            
            sentence.append(token)
            labels.append(label)
        sentence = ' '.join(sentence)
        labels = ','.join(labels)
        df.loc[len(df)] = {'sentence':sentence, 'word_labels':labels}
    
    print("Removed Entities: ", removed)
    print("Kept Entities: ", kept)
#    df.to_csv(output_path, header=True)
    return df

In [5]:
filepath = '../corpus/matscholar/train.txt'
#output_path = '../corpus/matscholar/matscholar_train_bio_full.csv'
output_path = ''
df = convert_matscholar_to_bio(filepath, output_path, 0.4)

['MPam', '1/2'] O
['MPam', '1/2'] O
Removed Entities:  12250
Kept Entities:  8136


In [81]:
for index, row in df.iterrows():
    labels = row['word_labels'].split(',')
    labels = [l for l in labels if l not in ['B','I','O']]
    if len(labels) != 0:
        print(labels)
        print(row['sentence'])